In [36]:
from toggl.TogglPy import Toggl
from dataclasses import dataclass
from typing import List

In [103]:
def convertMillis(millis):
    """
    Takes milliseconds and returns triple seconds, minutes, hours
    """
    seconds, milliseconds = divmod(millis,1000)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    seconds = seconds + milliseconds/1000
    return hours, minutes, seconds

In [3]:
token = "<token>"
toggl = Toggl()
toggl.setAPIKey(token)

In [1]:
# toggl.getWorkspaces()

In [19]:
defaultWsId = 4039412

In [96]:
since = "2020-03-01"
until = "2020-03-29"

dataFilter = {
    "workspace_id": defaultWsId,
    "since": since,
    "until": until
}
report = toggl.getSummaryReport(dataFilter)
report

{'total_grand': 128557000,
 'total_billable': None,
 'total_currencies': [{'currency': None, 'amount': None}],
 'data': [{'id': 157769715,
   'title': {'project': 'Other',
    'client': None,
    'color': '0',
    'hex_color': '#a01aa5'},
   'time': 2364000,
   'total_currencies': [{'currency': None, 'amount': None}],
   'items': [{'title': {'time_entry': 'finance stuff agreement with Pierrine'},
     'time': 2364000,
     'cur': None,
     'sum': None,
     'rate': None}]},
  {'id': 157719994,
   'title': {'project': 'Poll Bot',
    'client': None,
    'color': '0',
    'hex_color': '#06aaf5'},
   'time': 102272000,
   'total_currencies': [{'currency': None, 'amount': None}],
   'items': [{'title': {'time_entry': 'Automatic stats sending when all users voted'},
     'time': 7958000,
     'cur': None,
     'sum': None,
     'rate': None},
    {'title': {'time_entry': 'Berlin, integration tests'},
     'time': 50400000,
     'cur': None,
     'sum': None,
     'rate': None},
    {'title

In [105]:
totalWorkedInMillis = report['total_grand']

In [106]:
def millisToText(millis: int) -> str:
    hours, minutes, seconds = convertMillis(millis)
    return f"{hours} hours {minutes} minutes"

In [107]:
@dataclass
class Topic:
    name: str
    time: int
    
    def toString(self) -> str:
        return f"{self.name} - {millisToText(self.time)}"

@dataclass
class Project:
    name: str
    time: int
    topics: List[Topic]
    
    def toString(self) -> str:
        topics = "\n".join([f"|____ {x.toString()}\n" for x in self.topics])
        return f"{self.name} - total time - {millisToText(self.time)}\n{topics}"

In [108]:
def getDataForTopic(topic) -> Topic:
    name = topic['title']['time_entry']
    time = topic['time']
    return Topic(name, time)

def getDataForProject(project) -> Project:
    name = project['title']['project']
    time = project['time']
    topics = [getDataForTopic(x) for x in project['items']]
    return Project(name, time, topics)

In [109]:
projects = [getDataForProject(x) for x in report['data']]
projectsStringReport = "\n".join([x.toString() for x in projects])

In [110]:
finalReport = f"Total time worked since {since} until {until}: {millisToText(totalWorkedInMillis)}\nProjects report:\n{projectsStringReport}"
print(finalReport)

Total time worked since 2020-03-01 until 2020-03-29: 35 hours 42 minutes
Projects report:
Other - total time - 0 hours 39 minutes
|____ finance stuff agreement with Pierrine - 0 hours 39 minutes

Poll Bot - total time - 28 hours 24 minutes
|____ Automatic stats sending when all users voted - 2 hours 12 minutes

|____ Berlin, integration tests - 14 hours 0 minutes

|____ Database layer, Docker Compose setup - 9 hours 0 minutes

|____ fix logging - 0 hours 29 minutes

|____ user can  have only one vote - 2 hours 42 minutes

Simulation Framework - total time - 6 hours 38 minutes
|____ simulation framework - lithium onboarding - 6 hours 38 minutes

